In [ ]:
import sys, os
#sys.path.append(os.getcwd())
sys.path.append(os.path.abspath('..'))

import pandas as pd
import os
from user_profile import get_project_paths

paths = get_project_paths()
processed = paths['data_processed']

df1 = pd.read_csv(os.path.join(processed, 'df1_common_indicator.csv'))
df2 = pd.read_csv(os.path.join(processed, 'df2_common_track.csv'))
df3p = pd.read_csv(os.path.join(processed, 'df3_common_projections.csv'))

# Filter indicators for ANC4/SBA and years 2018-2022
filtered = df1[
    df1['INDICATOR:Indicator'].str.startswith(('MNCH_ANC4:', 'MNCH_SAB:')) &
    df1['TIME_PERIOD:Time period'].between(2018, 2022)
].copy()
filtered['Indicator'] = filtered['INDICATOR:Indicator'].str.extract(r'^(MNCH_[^:]+)')
filtered['Value'] = pd.to_numeric(filtered['OBS_VALUE:Observation Value'], errors='coerce')
filtered['Year'] = pd.to_numeric(filtered['TIME_PERIOD:Time period'], errors='coerce')

# Keep latest per country-indicator
latest = (
    filtered.sort_values(['ISO3Code', 'Indicator', 'Year'], ascending=[True, True, False])
    .drop_duplicates(subset=['ISO3Code', 'Indicator'])
)

births = df3p[df3p['Year'] == 2022][['ISO3Code', 'Births (thousands)']].copy()
births['Births'] = pd.to_numeric(births['Births (thousands)'], errors='coerce') * 1000
births = births[['ISO3Code', 'Births']]

# Merge
merged = latest.merge(births, on='ISO3Code', how='left')
merged = merged.merge(df2[['ISO3Code', 'Status.U5MR']], on='ISO3Code', how='left')
merged['Status.U5MR'] = merged['Status.U5MR'].str.strip().str.lower()
merged['Group'] = merged['Status.U5MR'].map(lambda x: 'On-track' if x in ['achieved', 'on-track'] else 'Off-track')
merged.dropna(subset=['Group', 'Births'], inplace=True)

pivot_df = merged.pivot_table(
    index=['ISO3Code', 'Group', 'Births'],
    columns='Indicator',
    values='Value',
    aggfunc='first'
).reset_index()

# Save merged dataset
processed = paths['data_processed']
pivot_df.to_csv(os.path.join(processed, 'merged_data.csv'), index=False)
print("Merged data saved to data/processed/merged_data.csv")

# Weighted average

def weighted_avg(df, col):
    df = df.dropna(subset=[col])
    return (df[col] * df['Births']).sum() / df['Births'].sum()

results = []
for group, gdf in pivot_df.groupby('Group'):
    anc4 = weighted_avg(gdf, 'MNCH_ANC4')
    sba = weighted_avg(gdf, 'MNCH_SAB')
    results.append({'Group': group, 'Weighted_ANC4': round(anc4, 2), 'Weighted_SBA': round(sba, 2)})

results_df = pd.DataFrame(results)
print(results_df)

# Save for reporting
outputs = paths['data_outputs']
results_df.to_csv(os.path.join(outputs, 'coverage_summary.csv'), index=False)
